In [61]:
import pickle as pkl
import os
import pandas as pd

path = os.getcwd()
data_name = 'SMD'
data_file = os.path.join(path, 'processed', data_name)
train_data_ = open(os.path.join(data_file, 'machine-1-1_train.pkl'),'rb')
train_data = pd.DataFrame(pkl.load(train_data_))
test_data_ = open(os.path.join(data_file, 'machine-1-1_test.pkl'),'rb')
test_data = pd.DataFrame(pkl.load(test_data_))

In [62]:
test_data_label_ = open(os.path.join(data_file, 'machine-1-1_test_label.pkl'),'rb')
test_data_label = pd.DataFrame(pkl.load(test_data_label_),columns=['label'])

In [63]:
anomaly_num = len(test_data_label[test_data_label.label==1])
anomaly_num / len(test_data_label)

0.09459601811861372

In [64]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_train_data = pd.DataFrame(scaler.fit_transform(train_data))
normalized_test_data = pd.DataFrame(scaler.fit_transform(test_data))
normalized_test_data

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.065217,0.065678,0.068156,0.073256,0.0,0.925287,0.269886,0.0,0.031081,0.000000,...,0.0,0.004317,0.068036,0.048893,0.000386,0.000023,0.064432,0.064500,0.0,0.0
1,0.076087,0.080508,0.073742,0.075582,0.0,0.922414,0.270833,0.0,0.031081,0.000122,...,0.0,0.004317,0.070020,0.050437,0.000386,0.000011,0.065228,0.065224,0.0,0.0
2,0.065217,0.064619,0.069273,0.073256,0.0,0.919540,0.270833,0.0,0.030940,0.000366,...,0.0,0.004317,0.069684,0.055069,0.000386,0.000034,0.067111,0.067178,0.0,0.0
3,0.076087,0.048729,0.061452,0.069768,0.0,0.919540,0.268940,0.0,0.027250,0.000244,...,0.0,0.005756,0.073253,0.051467,0.000000,0.000023,0.066676,0.066744,0.0,0.0
4,0.076087,0.051907,0.060335,0.069768,0.0,0.925287,0.269886,0.0,0.030940,0.000244,...,0.0,0.004317,0.070932,0.051467,0.000386,0.000011,0.066604,0.066671,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.065217,0.051907,0.045810,0.046512,0.0,0.896551,0.252841,0.0,0.043571,0.000244,...,0.0,0.027339,0.046733,0.040144,0.000000,0.000011,0.042931,0.043000,0.0,0.0
28475,0.054347,0.025424,0.036871,0.043024,0.0,0.893678,0.252841,0.0,0.032501,0.000000,...,0.0,0.027339,0.047438,0.048893,0.000000,0.000045,0.046550,0.046619,0.0,0.0
28476,0.054347,0.080508,0.048044,0.046512,0.0,0.896551,0.253788,0.0,0.026114,0.000611,...,0.0,0.027339,0.046797,0.040144,0.000386,0.000034,0.043003,0.043000,0.0,0.0
28477,0.054347,0.056144,0.045810,0.045350,0.0,0.890805,0.252841,0.0,0.033210,0.000122,...,0.0,0.027339,0.041884,0.043232,0.000000,0.000034,0.039890,0.039959,0.0,0.0


In [65]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca_train_data = pca.fit_transform(normalized_train_data)
pca_test_data = pca.fit_transform(normalized_test_data)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.6472445  0.23495755 0.03814477 0.01958583 0.01584375]
[67.19858  40.487465 16.313358 11.689527 10.513681]


## Ensemble

In [183]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import metrics

from pyod.models.combination import aom, moa, average, maximization, majority_vote
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

In [184]:
def pred_labels(y_test, test_scores):
    fpr, tpr, thresholds = roc_curve(y_test, test_scores)
    cutoff = thresholds[np.argmax(tpr - fpr)]
    pred_label = []
    for each in test_scores:
        if each > cutoff:
            pred_label.append(1)
        else:
            pred_label.append(0)    
    return pred_label

In [185]:
def cal_eval(label, pred_label, scores):
    pr = round(metrics.precision_score(label, pred_label, average='macro'), ndigits=4)
    re = round(metrics.recall_score(label, pred_label, average='macro'), ndigits=4)
    f1 = round(metrics.f1_score(label, pred_label, average='macro'), ndigits=4)
    roc = round(roc_auc_score(label, scores), ndigits=4)
    return pr,re,f1,roc

In [186]:
steps = 100
X_train = pca_test_data[:-steps]
X_test = pca_test_data[:-steps]
y_train = test_data_label[steps:]
y_test = test_data_label[steps:]

In [187]:
outliers_fraction = np.count_nonzero(y_train) / len(y_train)
outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
random_state = np.random.RandomState(42)
classifiers = {
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
    }

n_clf = len(classifiers)
train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])
p_labels = np.zeros([X_test.shape[0], n_clf])

i = 0
train_duration = 0
test_duration = 0
for clf_name, clf in classifiers.items():
    t0 = time()
    clf.fit(X_train)
    t1 = time()
    test_sccore = clf.decision_function(X_test)
    t_t = time()
    test_mean_score = np.nanmean(test_scores)
    test_scores[np.isnan(test_scores)] = test_mean_score
    test_scores[:, i] = test_sccore
    p_labels[:, i] = pred_labels(y_test, test_sccore)
    i += 1
    print('Base detector %i is fitted for prediction' % i)
    train_duration += round(t1 - t0, ndigits=4)
    test_duration += round(t_t - t1, ndigits=4)
    
# standardize test score
test_scores_norm = standardizer(test_scores)
print(train_duration)

Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
Base detector 4 is fitted for prediction
26.2735


## Keras

In [188]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import imdb
from time import time

#train_x, test_x, train_y, test_y = train_test_split(test_scores_norm, y_test, test_size=0.5, random_state=random_state)
train_x = test_scores_norm[:18000]
test_x = test_scores_norm[18000:] 
train_y = y_test[:18000]
test_y = y_test[18000:]

#train_x, test_x, train_y, test_y = train_test_split(test_scores_norm[:,3], y_test, test_size=0.5, random_state=random_state)

In [189]:
def ensemble_mlp():
    df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
    enm_mlp_df = pd.DataFrame(columns=df_columns)

    t2 = time()
    model = models.Sequential()
    # Input - Layer
    model.add(layers.Dense(4, activation = "relu", input_shape=(len(train_y),4)))
    model.add(layers.Dense(20, activation = "relu"))
    model.add(layers.Dense(20, activation = "relu"))
    # Output- Layer
    model.add(layers.Dense(1, activation = "sigmoid"))
    model.summary()
    # compiling the model
    model.compile(
        optimizer = "adam",
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )
    results = model.fit(
     train_x, train_y,
     epochs= 100,
     batch_size = 20,
     validation_data = (train_x, train_y)
    )
    t3 = time()
    #print(results.history['val_accuracy'])
    #pred_test = model.predict(test_scores_norm[:,3]).reshape(len(test_scores_norm),1)
    pred_test = model.predict(test_scores_norm)
    #pred_test = model.predict(test_x)
    t4 = time()
    train_time = t3 - t2
    test_time = t4 - t3
    train_time_mlp =  round(train_duration+train_time, ndigits=4)
    test_time_mlp = round(train_duration+test_time, ndigits=4)
    
    pred_nn = []
    for each in pred_test:
        if each[0] > 0.5:
            pred_nn.append(1)
        else:
            pred_nn.append(0)
    pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(y_test, pred_nn, pred_test)
    #pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(test_y, pred_nn, pred_test)
    enm_mlp = pd.DataFrame([pr_mlp, re_mlp, f1_mlp, roc_mlp, train_time_mlp, test_time_mlp]).transpose()
    enm_mlp.columns = df_columns
    enm_mlp_df = pd.concat([enm_mlp_df, enm_mlp], axis=0)
    return enm_mlp_df 

In [190]:
df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
t_enm_mlp_df = pd.DataFrame(columns=df_columns)
for i in range(10):
    print(i)
    enm_mlp = ensemble_mlp()
    t_enm_mlp_df = pd.concat([t_enm_mlp_df, enm_mlp], axis=0, ignore_index=True)

0
Model: "sequential_150"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_600 (Dense)            (None, 18000, 4)          20        
_________________________________________________________________
dense_601 (Dense)            (None, 18000, 20)         100       
_________________________________________________________________
dense_602 (Dense)            (None, 18000, 20)         420       
_________________________________________________________________
dense_603 (Dense)            (None, 18000, 1)          21        
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
900/900 [==============================] - 1s 1ms/step - loss: 0.2561 - accuracy: 0.9185 - val_loss: 0.1905 - val_accuracy: 0.9356
Epoch 2/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1873 - accuracy: 0.9352 - val_

1
Model: "sequential_151"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_604 (Dense)            (None, 18000, 4)          20        
_________________________________________________________________
dense_605 (Dense)            (None, 18000, 20)         100       
_________________________________________________________________
dense_606 (Dense)            (None, 18000, 20)         420       
_________________________________________________________________
dense_607 (Dense)            (None, 18000, 1)          21        
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
900/900 [==============================] - 2s 2ms/step - loss: 0.2340 - accuracy: 0.9370 - val_loss: 0.1919 - val_accuracy: 0.9373
Epoch 2/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1855 - accuracy: 0.9369 - val_

Epoch 99/100
900/900 [==============================] - 2s 2ms/step - loss: 0.1461 - accuracy: 0.9529 - val_loss: 0.1431 - val_accuracy: 0.9528
Epoch 100/100
900/900 [==============================] - 2s 2ms/step - loss: 0.1460 - accuracy: 0.9522 - val_loss: 0.1441 - val_accuracy: 0.9524
2
Model: "sequential_152"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_608 (Dense)            (None, 18000, 4)          20        
_________________________________________________________________
dense_609 (Dense)            (None, 18000, 20)         100       
_________________________________________________________________
dense_610 (Dense)            (None, 18000, 20)         420       
_________________________________________________________________
dense_611 (Dense)            (None, 18000, 1)          21        
Total params: 561
Trainable params: 561
Non-trainable params: 0
___________________________

Epoch 97/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1441 - accuracy: 0.9526 - val_loss: 0.1435 - val_accuracy: 0.9532
Epoch 98/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1444 - accuracy: 0.9518 - val_loss: 0.1426 - val_accuracy: 0.9524
Epoch 99/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1445 - accuracy: 0.9527 - val_loss: 0.1430 - val_accuracy: 0.9528
Epoch 100/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1445 - accuracy: 0.9517 - val_loss: 0.1435 - val_accuracy: 0.9527
3
Model: "sequential_153"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_612 (Dense)            (None, 18000, 4)          20        
_________________________________________________________________
dense_613 (Dense)            (None, 18000, 20)         100       
_________________________________________________________________
d

Epoch 95/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1470 - accuracy: 0.9527 - val_loss: 0.1457 - val_accuracy: 0.9527
Epoch 96/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1470 - accuracy: 0.9528 - val_loss: 0.1458 - val_accuracy: 0.9531
Epoch 97/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1469 - accuracy: 0.9529 - val_loss: 0.1460 - val_accuracy: 0.9531
Epoch 98/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1470 - accuracy: 0.9525 - val_loss: 0.1449 - val_accuracy: 0.9542
Epoch 99/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1468 - accuracy: 0.9527 - val_loss: 0.1458 - val_accuracy: 0.9542
Epoch 100/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1469 - accuracy: 0.9529 - val_loss: 0.1463 - val_accuracy: 0.9535
4
Model: "sequential_154"
_________________________________________________________________
Layer (type)                 Output Shape  

...

Epoch 91/100
900/900 [==============================] - 2s 2ms/step - loss: 0.1457 - accuracy: 0.9536 - val_loss: 0.1437 - val_accuracy: 0.9542
Epoch 92/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1456 - accuracy: 0.9526 - val_loss: 0.1456 - val_accuracy: 0.9539
Epoch 93/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1458 - accuracy: 0.9531 - val_loss: 0.1462 - val_accuracy: 0.9531
Epoch 94/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1462 - accuracy: 0.9530 - val_loss: 0.1445 - val_accuracy: 0.9536
Epoch 95/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1456 - accuracy: 0.9532 - val_loss: 0.1458 - val_accuracy: 0.9529
Epoch 96/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1458 - accuracy: 0.9536 - val_loss: 0.1440 - val_accuracy: 0.9536
Epoch 97/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1452 - accuracy: 0.9534 - val_loss: 0.1461 - val_accuracy:

Epoch 89/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1466 - accuracy: 0.9532 - val_loss: 0.1442 - val_accuracy: 0.9553
Epoch 90/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1460 - accuracy: 0.9542 - val_loss: 0.1441 - val_accuracy: 0.9556
Epoch 91/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1462 - accuracy: 0.9531 - val_loss: 0.1447 - val_accuracy: 0.9541
Epoch 92/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1457 - accuracy: 0.9537 - val_loss: 0.1445 - val_accuracy: 0.9535
Epoch 93/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1462 - accuracy: 0.9535 - val_loss: 0.1438 - val_accuracy: 0.9546
Epoch 94/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1455 - accuracy: 0.9538 - val_loss: 0.1440 - val_accuracy: 0.9550
Epoch 95/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1459 - accuracy: 0.9538 - val_loss: 0.1441 - val_accuracy:

Epoch 87/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1485 - accuracy: 0.9518 - val_loss: 0.1470 - val_accuracy: 0.9506
Epoch 88/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1485 - accuracy: 0.9510 - val_loss: 0.1474 - val_accuracy: 0.9518
Epoch 89/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1486 - accuracy: 0.9502 - val_loss: 0.1470 - val_accuracy: 0.9516
Epoch 90/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1490 - accuracy: 0.9509 - val_loss: 0.1478 - val_accuracy: 0.9516
Epoch 91/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1482 - accuracy: 0.9519 - val_loss: 0.1534 - val_accuracy: 0.9516
Epoch 92/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1484 - accuracy: 0.9504 - val_loss: 0.1485 - val_accuracy: 0.9510
Epoch 93/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1489 - accuracy: 0.9509 - val_loss: 0.1474 - val_accuracy:

Epoch 85/100
900/900 [==============================] - 2s 2ms/step - loss: 0.1449 - accuracy: 0.9542 - val_loss: 0.1431 - val_accuracy: 0.9546
Epoch 86/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1441 - accuracy: 0.9537 - val_loss: 0.1434 - val_accuracy: 0.9536
Epoch 87/100
900/900 [==============================] - 2s 2ms/step - loss: 0.1445 - accuracy: 0.9542 - val_loss: 0.1440 - val_accuracy: 0.9538
Epoch 88/100
900/900 [==============================] - 1s 2ms/step - loss: 0.1440 - accuracy: 0.9544 - val_loss: 0.1439 - val_accuracy: 0.9537
Epoch 89/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1445 - accuracy: 0.9536 - val_loss: 0.1435 - val_accuracy: 0.9541
Epoch 90/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1439 - accuracy: 0.9542 - val_loss: 0.1425 - val_accuracy: 0.9552
Epoch 91/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1441 - accuracy: 0.9540 - val_loss: 0.1418 - val_accuracy:

Epoch 83/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1496 - accuracy: 0.9526 - val_loss: 0.1502 - val_accuracy: 0.9534
Epoch 84/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1492 - accuracy: 0.9527 - val_loss: 0.1468 - val_accuracy: 0.9532
Epoch 85/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1491 - accuracy: 0.9536 - val_loss: 0.1480 - val_accuracy: 0.9534
Epoch 86/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1494 - accuracy: 0.9523 - val_loss: 0.1486 - val_accuracy: 0.9524
Epoch 87/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1493 - accuracy: 0.9531 - val_loss: 0.1491 - val_accuracy: 0.9518
Epoch 88/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1495 - accuracy: 0.9525 - val_loss: 0.1485 - val_accuracy: 0.9527
Epoch 89/100
900/900 [==============================] - 1s 1ms/step - loss: 0.1486 - accuracy: 0.9536 - val_loss: 0.1468 - val_accuracy:

In [191]:
print('Ensemble mlp')
metrics = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_mlp = []
std_mlp = []
for each in metrics:
    mean_mlp.append(round(t_enm_mlp_df[each].mean(), ndigits=4))
    std_mlp.append(round(t_enm_mlp_df[each].std(), ndigits=4))
print(mean_mlp)
print(std_mlp)
t_enm_mlp_df

Ensemble mlp
[0.8181, 0.671, 0.7157, 0.835, 151.533, 26.7175]
[0.0167, 0.0163, 0.0142, 0.0078, 16.3481, 0.1001]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.8223,0.6835,0.7288,0.8301,135.5692,26.7277
1,0.8514,0.6364,0.6860,0.8301,164.3996,26.7141
2,0.8080,0.6892,0.7304,0.8421,141.2668,26.6552
3,0.8221,0.6736,0.7198,0.8513,138.2992,26.6641
4,0.8026,0.6581,0.7016,0.8226,149.7003,26.9636
5,0.8223,0.6754,0.7215,0.8363,177.9546,26.6963
6,0.8104,0.6689,0.7131,0.8376,164.3973,26.6874
7,0.8322,0.6576,0.7061,0.8347,132.1192,26.6355
8,0.7908,0.6844,0.7221,0.8308,141.2524,26.8011
9,0.8192,0.6833,0.7279,0.8348,170.3713,26.6296
